# Colab Demo Code

The demo code is inspired from the XMem Colab Demo code

In [2]:
%pip install -r requirements.txt

  Cloning https://github.com/cheind/py-thin-plate-spline to /tmp/pip-req-build-g4sh09vy
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-thin-plate-spline /tmp/pip-req-build-g4sh09vy
  Resolved https://github.com/cheind/py-thin-plate-spline to commit f6995795397118b7d0ac01aecd3f39ffbfad9dee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)


In [16]:
!nvidia-smi

import torch

if torch.cuda.is_available():
  print('Using GPU')
  device = 'cuda'
else:
  print('CUDA not available. Please connect to a GPU instance if possible.')
  device = 'cpu'

Mon Dec 16 23:43:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 33%   41C    P8    13W / 200W |      1MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Get XMem and install prerequisit

In [6]:
!git clone https://github.com/hkchengrex/XMem.git
%cd XMem
%pip install -r requirements.txt

Cloning into 'XMem'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 608 (delta 257), reused 215 (delta 215), pack-reused 258 (from 1)
Receiving objects: 100% (608/608), 263.42 KiB | 1.04 MiB/s, done.
Resolving deltas: 100% (359/359), done.
/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/XMem


/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  Cloning https://github.com/cheind/py-thin-plate-spline to /private/var/folders/c2/cdv0qsp57bjbb3ww0lrp80pm0000gn/T/pip-req-build-en53k3mv
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-thin-plate-spline /private/var/folders/c2/cdv0qsp57bjbb3ww0lrp80pm0000gn/T/pip-req-build-en53k3mv
  Resolved https://github.com/cheind/py-thin-plate-spline to commit f6995795397118b7d0ac01aecd3f39ffbfad9dee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 5.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for thinplate: filename=thinplate-1.0.0-py3-none-any.whl size=6703 sha256=d960053df916003ac7c345

# Basic setup


In [5]:
%cd XMem

[Errno 2] No such file or directory: 'XMem'
/mnt/data/home/vanessa_share/Jonas/Leprosy/XMem


In [6]:
%matplotlib inline

Please download the weights from the following link: https://drive.google.com/drive/folders/10gBdoFOz6xRkam8knoLj1Ed0J0yOceIr?usp=sharing and add the paths from the weights

In [11]:
!wget -O YOLOv8m-seg.pt https://drive.google.com/uc?id=1XIqW8nLwVnl_YbI6QesSppZq1nhcBKSf

--2024-12-16 23:37:47--  https://drive.google.com/uc?id=1XIqW8nLwVnl_YbI6QesSppZq1nhcBKSf
Resolving drive.google.com (drive.google.com)... 142.250.186.78, 2a00:1450:4001:828::200e
Connecting to drive.google.com (drive.google.com)|142.250.186.78|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1XIqW8nLwVnl_YbI6QesSppZq1nhcBKSf [following]
--2024-12-16 23:37:47--  https://drive.usercontent.google.com/download?id=1XIqW8nLwVnl_YbI6QesSppZq1nhcBKSf
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.186.129, 2a00:1450:4001:82a::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.186.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2373 (2.3K) [text/html]
Saving to: ‘YOLOv8m-seg.pt’

YOLOv8m-seg.pt      100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-12-16 23:37:48 (17.2 MB/s) - ‘YOLOv8m-seg.pt’ saved

In [41]:
# Insert here the path to the weights of YOLOv8
path_weights_YOLO_detection = "/home/vanessa_share/Jonas/Leprosy/YOLOv8m_detection.pt"
path_weights_YOLO_segmentation = "/home/vanessa_share/Jonas/Leprosy/YOLOv8m-seg.pt"
xmem_path_weights = "/home/vanessa_share/Jonas/Leprosy/xmem_weights.pt"

# Insert here the path to the data
data_path = "/home/vanessa_share/Jonas/Leprosy/new_test_7"

In [17]:
import os
from os import path
from argparse import ArgumentParser
import shutil

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

from inference.data.test_datasets import LongTestDataset, DAVISTestDataset, YouTubeVOSTestDataset
from inference.data.mask_mapper import MaskMapper
from model.network import XMem
from inference.inference_core import InferenceCore

from progressbar import progressbar

torch.set_grad_enabled(False)

# default configuration
xmem_config = {
    'top_k': 30,
    'mem_every': 5,
    'deep_update_every': -1,
    'enable_long_term': True,
    'enable_long_term_count_usage': True,
    'num_prototypes': 128,
    'min_mid_term_frames': 5,
    'max_mid_term_frames': 10,
    'max_long_term_elements': 10000,
}


network = XMem(xmem_config, xmem_path_weights).eval().to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

Configuration for YOLOv8. For all options, please refer to https://docs.ultralytics.com/modes/predict/#inference-arguments

In [42]:
yolo_config = {
    'conf': 0.70,
    'max_det': 1,
    'show': True,
}

yolo_seg_config = {
    'conf': 0.25,
    'max_det': 1,
    'bounding_box': False, 
    'show': False
}

In [43]:
import ultralytics

# Detecting the key frames with YOLOv8

In [56]:
from ultralytics import YOLO
from matplotlib import pyplot as plt

model = YOLO(path_weights_YOLO_detection)
prediction_detection = model(data_path, max_det=yolo_config['max_det'],conf=yolo_config['conf'], show=yolo_config['show'])
i = 0
list_pred_images = []
for pred in prediction_detection:
    box =pred.boxes
    if box.conf.numel() == 0:
        print("No boxes detected")
        continue
    
    img = pred.orig_img
    img = Image.fromarray(img)
    img.save(f"img{i}.jpg")
    list_pred_images.append(pred)
    pred.save(filename=f"result{i}.jpg")
    i += 1
detected_frames= [x.orig_img for x in list_pred_images if x.boxes.conf.numel() != 0]
print(list_pred_images)
#image = prediction['orig_img']


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

image 1/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_0.png: 320x640 (no detections), 142.9ms
image 2/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_1.png: 320x640 1 nerve, 139.4ms
image 3/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_10.png: 320x640 (no detections), 170.3ms
image 4/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_11.png: 320x640 1 nerve, 139.8ms
image 5/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_12.png: 320x640 1 nerve, 139.3ms
image 6/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_13.png: 320x640 1 nerve, 139.7ms
image 7/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_14.png: 320x640 1 nerve, 139.3ms
image 8/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_15.png: 320x640 1 nerve, 138.5ms
image 9/246 /home/vanessa_share/Jonas/Leprosy/new_test_7/044_MD_16.png: 320x640 1 nerve, 139

# Segment the images

In [25]:
from ultralytics import YOLO

model = YOLO(path_weights_YOLO_segmentation)
prediction = model(data_path, max_det=yolo_seg_config['max_det'],conf=yolo_seg_config['conf'], show=yolo_seg_config['show'])
image = prediction['orig_img']


segment



FileNotFoundError: No images or videos found in . Supported formats are:
images: {'jpeg', 'jpg', 'bmp', 'png', 'tif', 'webp', 'tiff', 'heic', 'mpo', 'pfm', 'dng'}
videos: {'gif', 'mpg', 'ts', 'asf', 'wmv', 'mpeg', 'webm', 'mp4', 'mov', 'm4v', 'avi', 'mkv'}

# Convert the YOLO output to a binary segmentation mask

The function needs the original image to create the new image

In [4]:
import cv2
import os

def denormalize_point(x:float, y:float, width:int, height:int):
    return int(x*width), int(y*height)

# Read contours from text file
def convert_contours_from_result(contour_normalized, width, height):
    contours = []
    for line in contour_normalized:
        points = line.strip().split(' ')
        contour = [(denormalize_point(float(points[i]), float(points[i+1]),width,height)) for i in range(1, len(points), 2)]
        contours.append(contour)
    return contours

# Create a blank image
def create_blank_image(width, height):
    return np.zeros((height, width), dtype=np.uint8)

# Draw filled contours on image
def draw_filled_contours(image, contours):
    for contour in contours:
        pts = np.array(contour, dtype=np.int32)
        cv2.fillPoly(image, [pts], 1)

# Save image
def save_image(image, file_path):
    cv2.imwrite(file_path, image)

def get_shape(image_path:str):
    shape = cv2.imread(image_path).shape
    height = shape[0]
    width = shape[1]
    return width, height

def convert_to_binary_segmentation_mask(original_width:int,original_height:int, contoursfile:str, output_file:str):
    
    width, height = original_width, original_height
    contours = convert_contours_from_result(contoursfile, width=width,height=height)
    
    blank_image = create_blank_image(width, height)
    draw_filled_contours(blank_image, contours)
    save_image(blank_image, output_file)
    return blank_image.np()


In [ ]:
mask = convert_to_binary_segmentation_mask(image.shape[1], image.shape[0], prediction['contours'], 'mask.png')

# Propagate the remaining frames

In [ ]:
import cv2
from inference.interact.interactive_utils import image_to_torch, index_numpy_to_one_hot_torch, torch_prob_to_numpy_mask, overlay_davis

torch.cuda.empty_cache()
num_objects = 1
processor = InferenceCore(network, config=xmem_config)
processor.set_all_labels(range(1, num_objects+1)) # consecutive labels
cap = cv2.VideoCapture(video_name)

# You can change these two numbers
frames_to_propagate = 417
visualize_every = 1

current_frame_index = 0

with torch.cuda.amp.autocast(enabled=True):
  while (cap.isOpened()):
    # load frame-by-frame
    _, frame = cap.read()
    if frame is None or current_frame_index > frames_to_propagate:
      break

    # convert numpy array to pytorch tensor format
    frame_torch, _ = image_to_torch(frame, device=device)
    if current_frame_index == 0:
      # initialize with the mask
      mask_torch = index_numpy_to_one_hot_torch(mask, num_objects+1).to(device)
      # the background mask is not fed into the model
      prediction = processor.step(frame_torch, mask_torch[1:])

    else:
      # propagate only
      prediction = processor.step(frame_torch)

    # argmax, convert to numpy
    prediction = torch_prob_to_numpy_mask(prediction)

 

    if current_frame_index % visualize_every == 0:
      print(current_frame_index)
      visualization = overlay_davis(frame, prediction)
      display(Image.fromarray(visualization))

    current_frame_index += 1